# **Next2You functionality for 2.4 GHz**

Codebase for the paper "Next2You: Robust Copresence Detection Based on Channel State Information" by Mikhail Fomichev, Luis F. Abanto-Leon, Max Stiegler, Alejandro Molina, Jakob Link, Matthias Hollick, in ACM Transactions on Internet of Things (2021).

In [ ]:
# Import necessary modules for loading the CSI data 
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np

## Choose the CSI data from the specific experiment to be worked on. 

In [2]:
# Choose from which experiment the CSI data needs to be loaded, for the description of experiments see the paper 
# and/or the CSI data release: https://doi.org/10.5281/zenodo.5592335

# experiments = [['Office/2400 MHz/18-02-19', 'Office/2400 MHz/19-02-19', 'Office/2400 MHz/19-02-19-night', 'Office/2400 MHz/20-02-19']]
# experiments = [['Urban Flat/2400 MHz']]
# experiments = [['Rural Flat/2400 MHz']]
# experiments = [['Parking Cars/2400 MHz']]
# experiments = [['Moving Cars/2400 MHz']]
# experiments = [['6P/2400 MHz']]
# experiments = [['Beacon/2400 MHz']]
experiments = [['Power/2400 MHz']]

## Normal training (e.g., Table 3 in the paper)

We load the CSI data from the specified experiment in a stratified sampling fashion, deleting the CSI data of irrelevant null subcarriers (cf. Section 5.2 in the paper). 


In [ ]:
# Delete CSI data of null subcarriers
def delete_null(data):
    if data.shape[1]==128:
        null_carriers = np.array([0,32,33,34,35,29,30,31])
        null_carriers = np.append(null_carriers, null_carriers+64)
        data = np.delete(data, null_carriers, 1)
    elif data.shape[1]==510:
        null_carriers = np.array([0,1,123,124,125,126,127,128,129,130,131,132,133])
        null_carriers = np.append(null_carriers, null_carriers+255)
        data = np.delete(data, null_carriers, 1)
    else:
        null_carriers = np.array([0,1,123,124,125,126,127,128,129,130,131,132,133,255])
        null_carriers = np.append(null_carriers, null_carriers+256)
        data = np.delete(data, null_carriers, 1)
    return data


# Load CSI data in a stratified sampling fashion
def check_and_write(data_arr, labels_arr, data0, labels0, data1, labels1, fold, previous):
  if not data0 is None:
    print('Adding to subsets (zeros): Collector ' + previous, str(data0.shape))
    p0 = np.random.permutation(data0.shape[0])
    data0 = data0[p0]
    labels0 = labels0[p0]
    step_size0 = data0.shape[0]//fold
  if not data1 is None:
    print('Adding to subsets (ones): Collector ' + previous, str(data1.shape))
    p1 = np.random.permutation(data1.shape[0])
    data1 = data1[p1]
    labels1 = labels1[p1]
    step_size1 = data1.shape[0]//fold
  # watch out if step size is zero (should not happen for our experiments)
  # p is used to shuffle one more time when adding stratums to subsets
  p = np.random.permutation(fold)
  if not not data_arr:
    for i in range(fold-1):
      print("Loading " + 'data_arr ' + str(p[i]))
      if not data0 is None:
        print('Zero part: Adding to ' + 'data_arr ' + str(p[i]) + '; Former shape: ' + str(data_arr[p[i]].shape), 'Adding: ' + str(data0[i*step_size0:(i+1)*step_size0].shape))
        data_arr[p[i]] = np.concatenate((data_arr[p[i]],data0[i*step_size0:(i+1)*step_size0]), axis=0)
        labels_arr[p[i]] = np.concatenate((labels_arr[p[i]],labels0[i*step_size0:(i+1)*step_size0]), axis=0)
      if not data1 is None:
        print('One part: Adding to ' + 'data_arr ' + str(p[i]) + '; Former shape: ' + str(data_arr[p[i]].shape), 'Adding: ' + str(data1[i*step_size1:(i+1)*step_size1].shape))
        data_arr[p[i]] = np.concatenate((data_arr[p[i]],data1[i*step_size1:(i+1)*step_size1]), axis=0)
        labels_arr[p[i]] = np.concatenate((labels_arr[p[i]],labels1[i*step_size1:(i+1)*step_size1]), axis=0)
    if not data0 is None:
      print('Zero part: Adding to ' + 'csi_' + str(p[fold-1]) + '; Former shape: ' + str(data_arr[p[fold-1]].shape), 'Adding: ' + str(data0[(fold-1)*step_size0:].shape))
      data_arr[p[fold-1]] = np.concatenate((data_arr[p[fold-1]],data0[(fold-1)*step_size0:]), axis=0)
      labels_arr[p[fold-1]] = np.concatenate((labels_arr[p[fold-1]],labels0[(fold-1)*step_size0:]), axis=0)
    if not data1 is None:
      print('One part: Following added to ' + 'csi_' + str(p[fold-1]) + '; Former shape: ' + str(data_arr[p[fold-1]].shape), 'Adding: ' + str(data1[(fold-1)*step_size1:].shape))
      data_arr[p[fold-1]] = np.concatenate((data_arr[p[fold-1]],data1[(fold-1)*step_size1:]), axis=0)
      labels_arr[p[fold-1]] = np.concatenate((labels_arr[p[fold-1]],labels1[(fold-1)*step_size1:]), axis=0)
  else:
    if not data0 is None and not data1 is None:
      for i in range(fold-1):
        data_arr.append(data0[i*step_size0:(i+1)*step_size0])
        labels_arr.append(labels0[i*step_size0:(i+1)*step_size0])
        data_arr[i] = np.concatenate((data_arr[i],data1[i*step_size1:(i+1)*step_size1]), axis=0)
        labels_arr[i] = np.concatenate((labels_arr[i],labels1[i*step_size1:(i+1)*step_size1]), axis=0)
        print('Writing csi_' + str(p[i]), str(data_arr[i].shape))
      data_arr.append(data0[(fold-1)*step_size0:])
      labels_arr.append(labels0[(fold-1)*step_size0:])
      data_arr[fold-1] = np.concatenate((data_arr[fold-1],data1[(fold-1)*step_size1:]), axis=0)
      labels_arr[fold-1] = np.concatenate((labels_arr[fold-1],labels1[(fold-1)*step_size1:]), axis=0)
      print('Writing csi_' + str(p[fold-1]), str(data_arr[fold-1].shape))
    elif not data0 is None:
      for i in range(fold-1):
        data_arr.append(data0[i*step_size0:(i+1)*step_size0])
        labels_arr.append(labels0[i*step_size0:(i+1)*step_size0])
      data_arr.append(data0[(fold-1)*step_size0:])
      labels_arr.append(labels0[(fold-1)*step_size0:])
    elif not data1 is None:
      for i in range(fold-1):
        data_arr.append(data1[i*step_size1:(i+1)*step_size1])
        labels_arr.append(labels1[i*step_size1:(i+1)*step_size1])
      data_arr.append(data1[(fold-1)*step_size1:])
      labels_arr.append(labels1[(fold-1)*step_size1:])


# Form training and test sets of CSI data, important:
# (1) On Google Drive the CSI data is supposed to be stored at: My Drive/csi/data/ (e.g., My Drive/csi/data/6P)
# (2) (Un-)comment lines 107–109 to use only CSI magnitudes or phases of the CSI data, by default both are used
def validation_sets_from_data(directories, fold, seed):
  np.random.seed(seed)
  for directory in directories:
    path = '/content/drive/My Drive/csi-zia/data/' + directory
    relpath = '/content/drive/My Drive/csi-zia/data/' + directory + '/'
    previous = 'None'
    data0 = None
    data1 = None
    labels0 = None
    labels1 = None
    for filename in sorted(os.listdir(path)):
      if filename.endswith('.txt') and 'csi' in filename:
        #check if file belongs to new collector:
        collector = filename.split('_')[0]
        if collector == '':
          collector = filename.split('_')[1]
        if not previous==collector:
          if not previous=='None':
            check_and_write(csi_data, csi_labels, data0, labels0, data1, labels1, fold, previous)
          data0 = None
          data1 = None
          previous=collector
        #handle new file:  
        new = delete_null(np.loadtxt(relpath + filename))
        new_labels = np.loadtxt(relpath + filename.replace('csi','labels')) 
        # new_flatten = new[:,:new.shape[1]//2]                                     # use only CSI magnitudes
        # new_flatten = new[:,new.shape[1]//2:]                                     # use only CSI phases
        new_flatten = new                                                           # use both CSI magnitudes and phases                                     
        if new_labels[0]==1: 
          if data1 is None:
            data1 = new_flatten
            labels1 = new_labels
          else:  
            data1 = np.concatenate((data1,new_flatten), axis=0)
            labels1 = np.concatenate((labels1,new_labels), axis=0)
        else:
          if data0 is None:
            data0 = new_flatten
            labels0 = new_labels
          else:  
            data0 = np.concatenate((data0,new_flatten), axis=0)
            labels0 = np.concatenate((labels0,new_labels), axis=0)
    check_and_write(csi_data, csi_labels, data0, labels0, data1, labels1, fold, previous)
  print('Data successfully rearranged.')


# *****************************************************************************#

# Forming the data by calling the above functions
csi_data = []       # CSI data
csi_labels = []     # CSI labels: 1 - for colocated devices, 0 - for non-colocated devices

# Iterate over the experiments (see above Office data is composed from multiple data sets), important:
# (1) We set the number of folds to 5, i.e., for 5-fold CV in 'validation_sets_from_data' and 
# (2) The seed to shuffle the data to 123, for reproducibility (same function)
for experiment in experiments:
  print(experiment)
  validation_sets_from_data(experiment, 5, 123)
  print(experiment[0].split('/')[0] + '/' + experiment[0].split('/')[1] + ' subsets finished') 

print(csi_data[0].shape, csi_data[1].shape, csi_data[2].shape, csi_data[3].shape, csi_data[4].shape)
print(csi_labels[0].shape, csi_labels[1].shape, csi_labels[2].shape, csi_labels[3].shape, csi_labels[4].shape)
print('')
print('')

# Forming the training and test datasets
for i in range(1):
  test_data = csi_data[i]
  test_labels = csi_labels[i]

  train_data = csi_data[(i+1)%5]
  train_labels = csi_labels[(i+1)%5]
  for k in range(5):
    if not k==i and not k==(i+1)%5:
      train_data = np.concatenate((train_data, csi_data[k]), axis=0)
      train_labels = np.concatenate((train_labels, csi_labels[k]), axis=0)
  print(train_data.shape, train_labels.shape)
  print(test_data.shape, test_labels.shape)

  # Normalizing CSI data using variance scaling
  mean = np.mean(train_data, axis=0)
  train_data = train_data - mean
  std = np.std(train_data, axis=0)
  train_data = train_data / std
  test_data = (test_data - mean) / std


**Bottom line in the above cell:** The 'train_data' and 'test_data' contain the stratified data from the folds in the ratio 80% to 20%, respectively. 

We use this setup for subsequent training to speed up the computation. For a 5-fold cross validation (CV) described in the paper, apply the below code; when dealing with individual models, we reported the results corresponding to the best model obtainable from the 5-fold CV.

The results when using the 80% to 20% ratio vs. the full 5-fold CV differ marginally (i.e., the former shows slightly lower Area Under the ROC Curve – AUC).

## Train a neural network model with Keras (cf. Figure 7 in the paper for the structure).

In [3]:
# Import necessary modules
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score


# Convert soft predictions to binary
def make_bin_predictions(soft_pred):
  # Create binary predicitions array
  bin_pred = np.zeros((soft_pred.shape[0]), dtype=np.int8)

  # Convert from soft predictions to binary predictions
  for i in range(soft_pred.shape[0]):
    if soft_pred[i,1] > soft_pred[i,0]:
      bin_pred[i] = 1

  return bin_pred


# Train a neural network model and make predictions on the data
def nn_model_train_predict(data, labels, test, test_labels, num_epochs=35):
  # Define a model
  model = keras.Sequential()
  model.add(keras.layers.Dense(500, input_shape=(data.shape[1],), activation=tf.nn.leaky_relu))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(300, activation=tf.nn.leaky_relu)) 
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(100, activation=tf.nn.leaky_relu)) 
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(20, activation=tf.nn.leaky_relu)) 
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(2, activation=tf.nn.softmax))
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  # Perform training
  model.fit(data, labels, verbose=2, epochs=num_epochs, batch_size=32)

  # Get training and test accuracies
  train_loss, train_acc = model.evaluate(data, labels)
  test_loss, test_acc = model.evaluate(test, test_labels)

  # Make training and test soft predictions
  train_pred_soft = model.predict(data)
  test_pred_soft = model.predict(test)

  # Convert soft predictions to binary predictions
  train_pred = make_bin_predictions(train_pred_soft)
  test_pred = make_bin_predictions(test_pred_soft)

  return train_acc, test_acc, train_pred, test_pred, model

Train on the CSI data (i.e., 80% to 20% setup) and make copresence predictions.

In [ ]:
train_acc, test_acc, train_pred, test_pred, model = nn_model_train_predict(train_data, train_labels, test_data, test_labels)

Display the results: accuracy and AUC.

In [ ]:
print('Accuracy: %.4f' % test_acc)
print('AUC: %.4f' % roc_auc_score(test_labels, test_pred))

Optionally you can save the resulting neural network model.

In [ ]:
# Where to save the model and its name
model_path = '/content/drive/My Drive/csi-zia/results/'
model_name = 'my-model'

# Save Keras model
model.save(model_path + model_name + '.h5')

And (again optionally) load it back on.

In [ ]:
# Function to load the model
from keras.models import load_model

# From where to load the model 
model_path = '/content/drive/My Drive/csi-zia/results/'
model_name = 'my-model'

# Loaded model
l_model = tf.keras.models.load_model(model_path + model_name + '.h5', 
                                   custom_objects={'leaky_relu': tf.nn.leaky_relu})

# Verify that the model is correct returing the same AUC
print('AUC = %.4f' % (roc_auc_score(test_labels, 
                                    make_bin_predictions(model.predict(test_data)))))

## Training using a 5-fold CV.

**Prerequisite:** Cell 'Normal training' must be execute beforehand. 

In [ ]:
# Lists storing accuracies and AUCs of cross-validated folds
cv_acc = []
cv_auc = []

# Loop: we have five folds
for i in range(0, 5):
  # Set the test data and labels: one fold
  test_data = csi_data[i]
  test_labels = csi_labels[i] 

  # Initialize train data and labels
  train_data = None
  train_labels = None

  # Set train data and lebels: four folds
  for j in range(0, 5):
    if j != i:
      if train_data is None:
        train_data = csi_data[j]
        train_labels = csi_labels[j]
      else:
        train_data = np.concatenate((train_data, csi_data[j]), axis=0)
        train_labels = np.concatenate((train_labels, csi_labels[j]), axis=0)

  print(train_data.shape, train_labels.shape)
  print(test_data.shape, test_labels.shape)

  # Normalize CSI data prior to training with variance scaling
  mean = np.mean(train_data, axis=0)
  train_data = train_data - mean
  std = np.std(train_data, axis=0)
  train_data = train_data / std
  test_data = (test_data - mean) / std

  # Set the number of epochs the neural network will be trained for
  n_epochs = 25

  # Do the training, prediction
  _, test_acc, _, test_pred, model = nn_model_train_predict(train_data, train_labels, test_data, test_labels, n_epochs)

  # Display accuracy and AUC
  print('Accuracy: %.4f' % test_acc)
  test_auc = roc_auc_score(test_labels, test_pred)
  print('AUC: %.4f' % test_auc)

  # Save accuracy and AUC
  cv_acc.append(test_acc)
  cv_auc.append(test_auc)
  
  # Optionally save 'model' as shown above for future use or computing EERs

Display the results: accuracy and AUC.

In [ ]:
print('Accuracy: %.4f' % np.mean(np.array(cv_acc)))
print('AUC: %.4f' % np.mean(np.array(cv_auc)))

## Prepare data for different time of day (tod) model training  in the Office scenario (cf. Section 6.2 in the paper).

**Note:** functions 'delete_null' and 'check_and_write' are the same as in the above 'Normal training' cell. However, I have decided to keep the cells self-contained. 

The resulting data split is in the form of 80% to 20% ratio, as described in the 'Normal training' cell. To run with the 5-fold CV, adapt the code from the 'Training using a 5-fold CV' cell. However, the results should be marginally different. 

In [ ]:
# Office data
experiments = ['Office/2400 MHz/18-02-19', 'Office/2400 MHz/19-02-19', 'Office/2400 MHz/19-02-19-night', 'Office/2400 MHz/20-02-19']

# Delete CSI data of null subcarriers
def delete_null(data):
    if data.shape[1]==128:
        null_carriers = np.array([0,32,33,34,35,29,30,31])
        null_carriers = np.append(null_carriers, null_carriers+64)
        data = np.delete(data, null_carriers, 1)
    elif data.shape[1]==510:
        null_carriers = np.array([0,1,123,124,125,126,127,128,129,130,131,132,133])
        null_carriers = np.append(null_carriers, null_carriers+255)
        data = np.delete(data, null_carriers, 1)
    else:
        null_carriers = np.array([0,1,123,124,125,126,127,128,129,130,131,132,133,255])
        null_carriers = np.append(null_carriers, null_carriers+256)
        data = np.delete(data, null_carriers, 1)
    return data


# Load CSI data in a stratified sampling fashion
def check_and_write(data_arr, labels_arr, data0, labels0, data1, labels1, fold, previous):
  if not data0 is None:
    print('Adding to subsets (zeros): Collector ' + previous, str(data0.shape))
    p0 = np.random.permutation(data0.shape[0])
    data0 = data0[p0]
    labels0 = labels0[p0]
    step_size0 = data0.shape[0]//fold
  if not data1 is None:
    print('Adding to subsets (ones): Collector ' + previous, str(data1.shape))
    p1 = np.random.permutation(data1.shape[0])
    data1 = data1[p1]
    labels1 = labels1[p1]
    step_size1 = data1.shape[0]//fold
  # watch out if step size is zero (should not happen for our experiments)
  # p is used to shuffle one more time when adding stratums to subsets
  p = np.random.permutation(fold)
  if not not data_arr:
    for i in range(fold-1):
      print("Loading " + 'data_arr ' + str(p[i]))
      if not data0 is None:
        print('Zero part: Adding to ' + 'data_arr ' + str(p[i]) + '; Former shape: ' + str(data_arr[p[i]].shape), 'Adding: ' + str(data0[i*step_size0:(i+1)*step_size0].shape))
        data_arr[p[i]] = np.concatenate((data_arr[p[i]],data0[i*step_size0:(i+1)*step_size0]), axis=0)
        labels_arr[p[i]] = np.concatenate((labels_arr[p[i]],labels0[i*step_size0:(i+1)*step_size0]), axis=0)
      if not data1 is None:
        print('One part: Adding to ' + 'data_arr ' + str(p[i]) + '; Former shape: ' + str(data_arr[p[i]].shape), 'Adding: ' + str(data1[i*step_size1:(i+1)*step_size1].shape))
        data_arr[p[i]] = np.concatenate((data_arr[p[i]],data1[i*step_size1:(i+1)*step_size1]), axis=0)
        labels_arr[p[i]] = np.concatenate((labels_arr[p[i]],labels1[i*step_size1:(i+1)*step_size1]), axis=0)
    if not data0 is None:
      print('Zero part: Adding to ' + 'csi_' + str(p[fold-1]) + '; Former shape: ' + str(data_arr[p[fold-1]].shape), 'Adding: ' + str(data0[(fold-1)*step_size0:].shape))
      data_arr[p[fold-1]] = np.concatenate((data_arr[p[fold-1]],data0[(fold-1)*step_size0:]), axis=0)
      labels_arr[p[fold-1]] = np.concatenate((labels_arr[p[fold-1]],labels0[(fold-1)*step_size0:]), axis=0)
    if not data1 is None:
      print('One part: Following added to ' + 'csi_' + str(p[fold-1]) + '; Former shape: ' + str(data_arr[p[fold-1]].shape), 'Adding: ' + str(data1[(fold-1)*step_size1:].shape))
      data_arr[p[fold-1]] = np.concatenate((data_arr[p[fold-1]],data1[(fold-1)*step_size1:]), axis=0)
      labels_arr[p[fold-1]] = np.concatenate((labels_arr[p[fold-1]],labels1[(fold-1)*step_size1:]), axis=0)
  else:
    if not data0 is None and not data1 is None:
      for i in range(fold-1):
        data_arr.append(data0[i*step_size0:(i+1)*step_size0])
        labels_arr.append(labels0[i*step_size0:(i+1)*step_size0])
        data_arr[i] = np.concatenate((data_arr[i],data1[i*step_size1:(i+1)*step_size1]), axis=0)
        labels_arr[i] = np.concatenate((labels_arr[i],labels1[i*step_size1:(i+1)*step_size1]), axis=0)
        print('Writing csi_' + str(p[i]), str(data_arr[i].shape))
      data_arr.append(data0[(fold-1)*step_size0:])
      labels_arr.append(labels0[(fold-1)*step_size0:])
      data_arr[fold-1] = np.concatenate((data_arr[fold-1],data1[(fold-1)*step_size1:]), axis=0)
      labels_arr[fold-1] = np.concatenate((labels_arr[fold-1],labels1[(fold-1)*step_size1:]), axis=0)
      print('Writing csi_' + str(p[fold-1]), str(data_arr[fold-1].shape))
    elif not data0 is None:
      for i in range(fold-1):
        data_arr.append(data0[i*step_size0:(i+1)*step_size0])
        labels_arr.append(labels0[i*step_size0:(i+1)*step_size0])
      data_arr.append(data0[(fold-1)*step_size0:])
      labels_arr.append(labels0[(fold-1)*step_size0:])
    elif not data1 is None:
      for i in range(fold-1):
        data_arr.append(data1[i*step_size1:(i+1)*step_size1])
        labels_arr.append(labels1[i*step_size1:(i+1)*step_size1])
      data_arr.append(data1[(fold-1)*step_size1:])
      labels_arr.append(labels1[(fold-1)*step_size1:])


# Form training and test sets of CSI data, important:
# (1) On Google Drive the CSI data is supposed to be stored at: My Drive/csi/data/ (e.g., My Drive/csi/data/Office/2400 MHz/18-02-19)
# (2) Load CSI data in a stratified sampling fashion per time of day, i.e., m - morning, a - afternoon, e - evening, n - night
def validation_sets_from_data(directory, fold, seed):
  path = '/content/drive/My Drive/csi-zia/data/' + directory
  relpath = '/content/drive/My Drive/csi-zia/data/' + directory + '/'
  previous = 'None'
  data0 = None
  data1 = None
  labels0 = None
  labels1 = None
  np.random.seed(seed)
  for filename in sorted(os.listdir(path)):
    if filename.endswith('.txt') and 'csi' in filename:
      #check if file belongs to new collector:
      collector = filename.split('_')[0]
      if collector == '':
        collector = filename.split('_')[1]
      if not previous==collector:
        if not previous=='None':
          if '-m-' in previous:
            check_and_write(m_data, m_labels, data0, labels0, data1, labels1, fold, previous)
          elif '-a-' in previous:
            check_and_write(a_data, a_labels, data0, labels0, data1, labels1, fold, previous)
          elif '-e-' in previous:
            check_and_write(e_data, e_labels, data0, labels0, data1, labels1, fold, previous)
          else:
            check_and_write(n_data, n_labels, data0, labels0, data1, labels1, fold, previous)
        data0 = None
        data1 = None
        previous=collector
      #handle new file:  
      new_labels = np.loadtxt(relpath + filename.replace('csi','labels'))
      new = np.loadtxt(relpath + filename)
      if new_labels[0]==1: 
        if data1 is None:
          data1 = np.loadtxt(relpath + filename)
          labels1 = np.loadtxt(relpath + filename.replace('csi','labels'))
        else:  
          data1 = np.concatenate((data1,new), axis=0)
          labels1 = np.concatenate((labels1,new_labels), axis=0)
      else:
        if data0 is None:
          data0 = np.loadtxt(relpath + filename)
          labels0 = np.loadtxt(relpath + filename.replace('csi','labels'))
        else:  
          data0 = np.concatenate((data0,new), axis=0)
          labels0 = np.concatenate((labels0,new_labels), axis=0)
  if '-m-' in previous:
    check_and_write(m_data, m_labels, data0, labels0, data1, labels1, fold, previous)
  elif '-a-' in previous:
    check_and_write(a_data, a_labels, data0, labels0, data1, labels1, fold, previous)
  elif '-e-' in previous:
    check_and_write(e_data, e_labels, data0, labels0, data1, labels1, fold, previous)
  else:
    check_and_write(n_data, n_labels, data0, labels0, data1, labels1, fold, previous)
  print('Data successfully rearranged.')


# *****************************************************************************#

# Forming the data by calling the above functions
m_data = [] # CSI data per time of day: morning, afternoon, evening, night
a_data = []
e_data = []
n_data = []

m_labels = [] # CSI labels: 1 - for colocated devices, 0 - for non-colocated devices
a_labels = []
e_labels = []
n_labels = []

# Iterate over the experiments (see above Office data is composed from multiple data sets), important:
# (1) We set the number of folds to 5, i.e., for 5-fold CV in 'validation_sets_from_data' and 
# (2) The seed to shuffle the data to 123, for reproducibility (same function)
for experiment in experiments:
  print(experiment)
  validation_sets_from_data(experiment, 5, 123)
  print(experiment + ' subsets finished')          

print(m_data[0].shape, m_data[1].shape, m_data[2].shape, m_data[3].shape, m_data[4].shape)
print(m_labels[0].shape, m_labels[1].shape, m_labels[2].shape, m_labels[3].shape, m_labels[4].shape)
print(a_data[0].shape, a_data[1].shape, a_data[2].shape, a_data[3].shape, a_data[4].shape)
print(a_labels[0].shape, a_labels[1].shape, a_labels[2].shape, a_labels[3].shape, a_labels[4].shape)
print(e_data[0].shape, e_data[1].shape, e_data[2].shape, e_data[3].shape, e_data[4].shape)
print(e_labels[0].shape, e_labels[1].shape, e_labels[2].shape, e_labels[3].shape, e_labels[4].shape)
print(n_data[0].shape, n_data[1].shape, n_data[2].shape, n_data[3].shape, n_data[4].shape)
print(n_labels[0].shape, n_labels[1].shape, n_labels[2].shape, n_labels[3].shape, n_labels[4].shape)
print('')
print('')

# Forming the training and test datasets for each time of day CSI data
for i in range(1):
  m_test_data = delete_null(m_data[i])
  a_test_data = delete_null(a_data[i])
  e_test_data = delete_null(e_data[i])
  n_test_data = delete_null(n_data[i])

  m_test_labels = m_labels[i]
  a_test_labels = a_labels[i]
  e_test_labels = e_labels[i]
  n_test_labels = n_labels[i]

  m_train_data = delete_null(m_data[(i+1)%5])
  a_train_data = delete_null(a_data[(i+1)%5])
  e_train_data = delete_null(e_data[(i+1)%5])
  n_train_data = delete_null(n_data[(i+1)%5])

  m_train_labels = m_labels[(i+1)%5]
  a_train_labels = a_labels[(i+1)%5]
  e_train_labels = e_labels[(i+1)%5]
  n_train_labels = n_labels[(i+1)%5]

  for k in range(5):
    if not k==i and not k==(i+1)%5:
      m_train_data = np.concatenate((m_train_data,delete_null(m_data[k])), axis=0)
      m_train_labels = np.concatenate((m_train_labels, m_labels[k]), axis=0)
      a_train_data = np.concatenate((a_train_data,delete_null(a_data[k])), axis=0)
      a_train_labels = np.concatenate((a_train_labels, a_labels[k]), axis=0)
      e_train_data = np.concatenate((e_train_data,delete_null(e_data[k])), axis=0)
      e_train_labels = np.concatenate((e_train_labels, e_labels[k]), axis=0)
      n_train_data = np.concatenate((n_train_data,delete_null(n_data[k])), axis=0)
      n_train_labels = np.concatenate((n_train_labels, n_labels[k]), axis=0)
  
  print(m_train_data.shape, m_train_labels.shape, a_train_data.shape, a_train_labels.shape, e_train_data.shape, e_train_labels.shape, n_train_data.shape, n_train_labels.shape)
  print(m_test_data.shape, m_test_labels.shape, a_test_data.shape, a_test_labels.shape, e_test_data.shape, e_test_labels.shape, n_test_data.shape, n_test_labels.shape)

  train_data = []
  test_data = []
  train_labels = []
  test_labels = []
  
  # Perform variance scaling normalization of CSI data
  mean = np.mean(m_train_data, axis=0)
  m_train_data = m_train_data - mean
  std = np.std(m_train_data, axis=0)
  m_train_data = m_train_data / std
  m_test_data = (m_test_data - mean) / std
  # train_data.append(m_train_data[:,:m_train_data.shape[1]//2])                  # Exclude CSI phase (just for illustrative purposes, e.g., for future use)
  # test_data.append(m_test_data[:,:m_train_data.shape[1]//2])
  train_data.append(m_train_data)
  test_data.append(m_test_data)               
  train_labels.append(m_train_labels)
  test_labels.append(m_test_labels)
  
  mean = np.mean(a_train_data, axis=0)
  a_train_data = a_train_data - mean
  std = np.std(a_train_data, axis=0)
  a_train_data = a_train_data / std
  a_test_data = (a_test_data - mean) / std
  # train_data.append(a_train_data[:,:m_train_data.shape[1]//2])                  # Exclude CSI phase (just for illustrative purposes, e.g., for future use)
  # test_data.append(a_test_data[:,:m_train_data.shape[1]//2])
  train_data.append(a_train_data)
  test_data.append(a_test_data)
  train_labels.append(a_train_labels)
  test_labels.append(a_test_labels)
  
  mean = np.mean(e_train_data, axis=0)
  e_train_data = e_train_data - mean
  std = np.std(e_train_data, axis=0)
  e_train_data = e_train_data / std
  e_test_data = (e_test_data - mean) / std
  # train_data.append(e_train_data[:,:m_train_data.shape[1]//2])                  # Exclude CSI phase (just for illustrative purposes, e.g., for future use)
  # test_data.append(e_test_data[:,:m_train_data.shape[1]//2])
  train_data.append(e_train_data)                       
  test_data.append(e_test_data)
  train_labels.append(e_train_labels)
  test_labels.append(e_test_labels)

  mean = np.mean(n_train_data, axis=0)
  n_train_data = n_train_data - mean
  std = np.std(n_train_data, axis=0)
  n_train_data = n_train_data / std
  n_test_data = (n_test_data - mean) / std
  # train_data.append(n_train_data[:,:m_train_data.shape[1]//2])                  # Exclude CSI phase (just for illustrative purposes, e.g., for future use)
  # test_data.append(n_test_data[:,:m_train_data.shape[1]//2])
  train_data.append(n_train_data)
  test_data.append(n_test_data)
  train_labels.append(n_train_labels)
  test_labels.append(n_test_labels)

Do the training on the time of day (tod) CSI data. 

In [ ]:
# List to store accuracies and predictions
tod_test_pred = []
tod_test_acc = []
tod_models = []

# Train models for each time of day
for i in range(len(train_data)):
  _, test_acc, _, test_pred, model = nn_model_train_predict(train_data[i], train_labels[i], test_data[i], test_labels[i], 25)
  tod_test_acc.append(test_acc)
  tod_test_pred.append(test_pred)
  tod_models.append(model)

Display the results: accuracy and AUC.



In [ ]:
# Print accuracy and AUC for each time of day model
for acc, pred, labels in zip(tod_test_acc, tod_test_pred, test_labels):
  print('Accuracy: %.4f' % acc)
  print('AUC: %.4f' % roc_auc_score(labels, pred))

## Phase sanitization (cf. Section 6.4 in the paper).

**Note:** functions 'delete_null' and 'check_and_write' are the same as in the above 'Normal training' cell. However, I have decided to keep the cells self-contained. 

The resulting data split is in the form of 80% to 20% ratio, as described in the 'Normal training' cell.


In [ ]:
# Delete CSI data of null subcarriers
def delete_null(data):
    if data.shape[1]==128:
        null_carriers = np.array([0,32,33,34,35,29,30,31])
        null_carriers = np.append(null_carriers, null_carriers+64)
        data = np.delete(data, null_carriers, 1)
    elif data.shape[1]==510:
        null_carriers = np.array([0,1,123,124,125,126,127,128,129,130,131,132,133])
        null_carriers = np.append(null_carriers, null_carriers+255)
        data = np.delete(data, null_carriers, 1)
    else:
        null_carriers = np.array([0,1,123,124,125,126,127,128,129,130,131,132,133,255])
        null_carriers = np.append(null_carriers, null_carriers+256)
        data = np.delete(data, null_carriers, 1)
    return data


# Load CSI data in a stratified sampling fashion
def check_and_write(data_arr, labels_arr, data0, labels0, data1, labels1, fold, previous):
  if not data0 is None:
    print('Adding to subsets (zeros): Collector ' + previous, str(data0.shape))
    p0 = np.random.permutation(data0.shape[0])
    data0 = data0[p0]
    labels0 = labels0[p0]
    step_size0 = data0.shape[0]//fold
  if not data1 is None:
    print('Adding to subsets (ones): Collector ' + previous, str(data1.shape))
    p1 = np.random.permutation(data1.shape[0])
    data1 = data1[p1]
    labels1 = labels1[p1]
    step_size1 = data1.shape[0]//fold
  # watch out if step size is zero (should not happen for our experiments)
  # p is used to shuffle one more time when adding stratums to subsets
  p = np.random.permutation(fold)
  if not not data_arr:
    for i in range(fold-1):
      print("Loading " + 'data_arr ' + str(p[i]))
      if not data0 is None:
        print('Zero part: Adding to ' + 'data_arr ' + str(p[i]) + '; Former shape: ' + str(data_arr[p[i]].shape), 'Adding: ' + str(data0[i*step_size0:(i+1)*step_size0].shape))
        data_arr[p[i]] = np.concatenate((data_arr[p[i]],data0[i*step_size0:(i+1)*step_size0]), axis=0)
        labels_arr[p[i]] = np.concatenate((labels_arr[p[i]],labels0[i*step_size0:(i+1)*step_size0]), axis=0)
      if not data1 is None:
        print('One part: Adding to ' + 'data_arr ' + str(p[i]) + '; Former shape: ' + str(data_arr[p[i]].shape), 'Adding: ' + str(data1[i*step_size1:(i+1)*step_size1].shape))
        data_arr[p[i]] = np.concatenate((data_arr[p[i]],data1[i*step_size1:(i+1)*step_size1]), axis=0)
        labels_arr[p[i]] = np.concatenate((labels_arr[p[i]],labels1[i*step_size1:(i+1)*step_size1]), axis=0)
    if not data0 is None:
      print('Zero part: Adding to ' + 'csi_' + str(p[fold-1]) + '; Former shape: ' + str(data_arr[p[fold-1]].shape), 'Adding: ' + str(data0[(fold-1)*step_size0:].shape))
      data_arr[p[fold-1]] = np.concatenate((data_arr[p[fold-1]],data0[(fold-1)*step_size0:]), axis=0)
      labels_arr[p[fold-1]] = np.concatenate((labels_arr[p[fold-1]],labels0[(fold-1)*step_size0:]), axis=0)
    if not data1 is None:
      print('One part: Following added to ' + 'csi_' + str(p[fold-1]) + '; Former shape: ' + str(data_arr[p[fold-1]].shape), 'Adding: ' + str(data1[(fold-1)*step_size1:].shape))
      data_arr[p[fold-1]] = np.concatenate((data_arr[p[fold-1]],data1[(fold-1)*step_size1:]), axis=0)
      labels_arr[p[fold-1]] = np.concatenate((labels_arr[p[fold-1]],labels1[(fold-1)*step_size1:]), axis=0)
  else:
    if not data0 is None and not data1 is None:
      for i in range(fold-1):
        data_arr.append(data0[i*step_size0:(i+1)*step_size0])
        labels_arr.append(labels0[i*step_size0:(i+1)*step_size0])
        data_arr[i] = np.concatenate((data_arr[i],data1[i*step_size1:(i+1)*step_size1]), axis=0)
        labels_arr[i] = np.concatenate((labels_arr[i],labels1[i*step_size1:(i+1)*step_size1]), axis=0)
        print('Writing csi_' + str(p[i]), str(data_arr[i].shape))
      data_arr.append(data0[(fold-1)*step_size0:])
      labels_arr.append(labels0[(fold-1)*step_size0:])
      data_arr[fold-1] = np.concatenate((data_arr[fold-1],data1[(fold-1)*step_size1:]), axis=0)
      labels_arr[fold-1] = np.concatenate((labels_arr[fold-1],labels1[(fold-1)*step_size1:]), axis=0)
      print('Writing csi_' + str(p[fold-1]), str(data_arr[fold-1].shape))
    elif not data0 is None:
      for i in range(fold-1):
        data_arr.append(data0[i*step_size0:(i+1)*step_size0])
        labels_arr.append(labels0[i*step_size0:(i+1)*step_size0])
      data_arr.append(data0[(fold-1)*step_size0:])
      labels_arr.append(labels0[(fold-1)*step_size0:])
    elif not data1 is None:
      for i in range(fold-1):
        data_arr.append(data1[i*step_size1:(i+1)*step_size1])
        labels_arr.append(labels1[i*step_size1:(i+1)*step_size1])
      data_arr.append(data1[(fold-1)*step_size1:])
      labels_arr.append(labels1[(fold-1)*step_size1:])


# Form training and test sets of CSI data, important:
# (1) On Google Drive the CSI data is supposed to be stored at: My Drive/csi/data/ (e.g., My Drive/csi/data/6P)
# (2) Here we use only CSI phases
def validation_sets_from_data(directories, fold, seed):
  np.random.seed(seed)
  for directory in directories:
    path = '/content/drive/My Drive/csi-zia/data/' + directory
    relpath = '/content/drive/My Drive/csi-zia/data/' + directory + '/'
    previous = 'None'
    data0 = None
    data1 = None
    labels0 = None
    labels1 = None
    for filename in sorted(os.listdir(path)):
      if filename.endswith('.txt') and 'csi' in filename:
        #check if file belongs to new collector:
        collector = filename.split('_')[0]
        if collector == '':
          collector = filename.split('_')[1]
        if not previous==collector:
          if not previous=='None':
            check_and_write(csi_phase, phase_labels, data0, labels0, data1, labels1, fold, previous)
          data0 = None
          data1 = None
          previous=collector
        #handle new file:  
        new = delete_null(np.loadtxt(relpath + filename))
        new_labels = np.loadtxt(relpath + filename.replace('csi','labels')) 
        new_flatten = new[:,new.shape[1]//2:]                                     # use only CSI phases
        if new_labels[0]==1: 
          if data1 is None:
            data1 = new_flatten
            labels1 = new_labels
          else:  
            data1 = np.concatenate((data1,new_flatten), axis=0)
            labels1 = np.concatenate((labels1,new_labels), axis=0)
        else:
          if data0 is None:
            data0 = new_flatten
            labels0 = new_labels
          else:  
            data0 = np.concatenate((data0,new_flatten), axis=0)
            labels0 = np.concatenate((labels0,new_labels), axis=0)
    check_and_write(csi_phase, phase_labels, data0, labels0, data1, labels1, fold, previous)
  print('Data successfully rearranged.')


# *****************************************************************************#

# Forming the data by calling the above functions: give explicit names for phase
csi_phase = []      # CSI phase data
phase_labels = []   # Phase labels: 1 - for colocated devices, 0 - for non-colocated devices

# Iterate over the experiments (see above Office data is composed from multiple data sets), important:
# (1) We set the number of folds to 5, i.e., for 5-fold CV in 'validation_sets_from_data' and 
# (2) The seed to shuffle the data to 123, for reproducibility (same function)
for experiment in experiments:
  print(experiment)
  validation_sets_from_data(experiment, 5, 123)
  print(experiment[0].split('/')[0] + '/' + experiment[0].split('/')[1] + ' subsets finished') 

print(csi_phase[0].shape, csi_phase[1].shape, csi_phase[2].shape, csi_phase[3].shape, csi_phase[4].shape)
print(phase_labels[0].shape, phase_labels[1].shape, phase_labels[2].shape, phase_labels[3].shape, phase_labels[4].shape)
print('')
print('')

# Store sanitized phases (for each fold similar to csi_phase above)
sanitized_phase = []

To perform *phase santitization* **execute** the below cell, otherwise if you want to use *raw phases*—**skip** this cell and proceed to the next one. 

In [ ]:
# Carrier indices for 2.4 GHz
carrier_idx = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 
               23, 24, 25, 26, 27, 28, -28, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, 
               -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]


def phase_sanitization(raw_phase):
    # Iterate over folds
    for fold in raw_phase:
        # Fold data (i.e., phases)
        fold_phase = []

        # Iterate over each CSI measurement in the fold
        for meas in fold:
            # Construct dict with carrier idx (keys) and phases (values)
            p = dict(zip(carrier_idx, meas))

            # Store values for one CSI measurement
            meas_phase = []
            
            # Iterate over p
            for k, v in p.items():
                a = (p[28] - p[-28]) / len(p) * k
                b = 1 / len(p) * np.sum(meas)
                meas_phase.append(v - a - b)

            # Add santized phases of one CSI measurement
            fold_phase.append(meas_phase)

        # Add sanitized phases of one fold
        sanitized_phase.append(np.array(fold_phase))

 
# Call 'phase_sanitization'
phase_sanitization(csi_phase)

Form the training and test sets.

In [ ]:
# Check if 'sanitized_phase' contains data, then we should work with it
if sanitized_phase:
  csi_phase = sanitized_phase

# Forming the training and test datasets
for i in range(1):
  test_data = csi_phase[i]
  test_labels = phase_labels[i]
  
  train_data = csi_phase[(i+1)%5]
  train_labels = phase_labels[(i+1)%5]
  for k in range(5):
    if not k==i and not k==(i+1)%5:
      train_data = np.concatenate((train_data, csi_phase[k]), axis=0)
      train_labels = np.concatenate((train_labels, phase_labels[k]), axis=0)
  print(train_data.shape, train_labels.shape)
  print(test_data.shape, test_labels.shape)

  # Normalizing CSI data using variance scaling
  mean = np.mean(train_data, axis=0)
  train_data = train_data - mean
  std = np.std(train_data, axis=0)
  train_data = train_data / std
  test_data = (test_data - mean) / std

Train on the CSI phases (i.e., 80% to 20% setup) and make copresence predictions.

In [ ]:
train_acc, test_acc, train_pred, test_pred, model = nn_model_train_predict(train_data, train_labels, test_data, test_labels)

Display the results: accuracy and AUC.

In [ ]:
print('Accuracy: %.4f' % test_acc)
print('AUC: %.4f' % roc_auc_score(test_labels, test_pred))

## Right for the Right Reasons (RRR) code taken from the original [paper](https://github.com/dtak/rrr). 

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

to_logprob = lambda L: L - tf.reduce_logsumexp(L, axis=1, keep_dims=True)

def one_hot(y):
  if len(y.shape) != 1:
    return y
  values = np.array(sorted(list(set(y))))
  return np.array([values == v for v in y], dtype=np.uint8)

class TensorflowPerceptron():
  def loss_function(self, l2_grads=1000, l1_grads=0, l2_params=0.0001):
    right_answer_loss = tf.reduce_sum(tf.multiply(self.y, -self.log_prob_ys))

    gradXes = tf.gradients(self.log_prob_ys, self.X)[0]
    A_gradX = tf.multiply(self.A, gradXes)
    right_reason_loss = 0
    if l1_grads > 0:
      right_reason_loss += l1_grads * tf.reduce_sum(tf.abs(A_gradX))
    if l2_grads > 0:
      right_reason_loss += l2_grads * tf.nn.l2_loss(A_gradX)

    small_params_loss = l2_params * tf.add_n([tf.nn.l2_loss(p) for p in self.W + self.b])

    return right_answer_loss + right_reason_loss + small_params_loss

  def optimizer(self, l2_grads=1000, l1_grads=0, l2_params=0.0001, learning_rate=0.001):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    return optimizer.minimize(self.loss_function(l2_grads=l2_grads, l1_grads=l1_grads, l2_params=l2_params))

  def fit(self, X, y, A=None,
      hidden_layers=[50,30], nonlinearity=tf.nn.relu, weight_sd=0.1,
      l2_grads=1000, l1_grads=0, l2_params=0.001,
      num_epochs=64, batch_size=256, learning_rate=0.001):

    y_org = y
    y = one_hot(y)
    y_dimensions = y.shape[1]
    x_dimensions = X.shape[1]
    num_examples = X.shape[0]
    if A is None:
      A = np.zeros((num_examples, x_dimensions))
    assert(num_examples == y.shape[0])
    assert(A.shape == X.shape)

    # set up network
    self.layer_sizes = [x_dimensions] + list(hidden_layers) + [y_dimensions]
    self.X = tf.placeholder("float", [None, x_dimensions], name="X")
    self.A = tf.placeholder("float", [None, x_dimensions], name="A")
    self.y = tf.placeholder("float", [None, y_dimensions], name="y")
    self.W = []
    self.b = []
    self.L = [self.X]
    for i in range(1, len(self.layer_sizes)):
      self.W.append(tf.Variable(tf.random_normal(self.layer_sizes[i-1:i+1], stddev=weight_sd), name='W{}'.format(i)))
      self.b.append(tf.Variable(tf.random_normal([self.layer_sizes[i]], stddev=weight_sd), name='b{}'.format(i)))
    for i, activation in enumerate([nonlinearity for _ in hidden_layers] + [to_logprob]):
      self.L.append(activation(tf.add(tf.matmul(self.L[i], self.W[i]), self.b[i])))

    # Set up optimization
    optimizer = self.optimizer(learning_rate=learning_rate, l2_grads=l2_grads, l1_grads=l1_grads, l2_params=l2_params)
    batch_size = min(batch_size, num_examples)
    num_batches = int(np.ceil(num_examples / batch_size))

    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      for i in range(num_epochs):
        for k in range(num_batches):
          idx = slice(k*batch_size, (k+1)*batch_size)
          sess.run(optimizer, feed_dict={self.X: X[idx], self.y: y[idx], self.A: A[idx]})
        print("Epoch {0} -> Accuracy: {1:.2f}%".format(i,self.score(X, y_org, from_session=True)*100))
      self.W_vals = [weights.eval() for weights in self.W]
      self.b_vals = [biases.eval() for biases in self.b]
  
  def _initialize_variables(self, sess):
    sess.run(tf.global_variables_initializer())
    for var, val in zip(self.W + self.b, self.W_vals + self.b_vals):
      sess.run(var.assign(val))

  @property
  def log_prob_ys(self):
    return self.L[-1]

  def input_gradients(self, X, y=None, log_scale=True):
    with tf.Session() as session:
      self._initialize_variables(session)
      probs = self.log_prob_ys
      if y is not None: probs = probs[:,y]
      if not log_scale: probs = tf.exp(probs)
      grads = tf.gradients(probs, self.X)[0].eval(feed_dict={self.X: X})
    return grads

  def largest_gradient_mask(self, X, cutoff=0.67, **kwargs):
    grads = self.input_gradients(X, **kwargs)
    return np.array([np.abs(g) > cutoff*np.abs(g).max() for g in grads])

  def predict_log_proba(self, X, from_session=False):  
    if not from_session:
      with tf.Session() as session:
        self._initialize_variables(session)
        log_probs = self.log_prob_ys.eval(feed_dict={self.X: X})
    else:
      log_probs = self.log_prob_ys.eval(feed_dict={self.X: X})
    return log_probs

  def predict(self, X, **kwargs):
    return np.argmax(self.predict_log_proba(X, **kwargs), axis=1)

  def predict_proba(self, X):
    return np.exp(self.predict_log_proba(X), axis=1)

  def score(self, X, y, **kwargs):
    return np.mean(self.predict(X, **kwargs) == y)


## Logic for applying the RRR method to find which features are considered important by a neural network in the copresence decision task (cf. Section 6.4 in the paper).

In [ ]:
# *************************** Core logic *******************************#
def get_relevant_features(model, train_set, test_set=None, cutoff=0.67):
  # Print current size of train and test sets
  if test_set is not None:
    print(train_set.shape)
    print(test_set.shape)
    print()

  # Obtain gradient mask for the train set
  mask = model.largest_gradient_mask(train_set, cutoff=cutoff)

  # Get mask for positive predictions
  pos_mask = mask

  # Count feature occurrence in the positive mask
  mask_count = np.sum(pos_mask, axis=0)

  # Store feature importance
  feature_import = {}

  # Iterate over features
  indices = np.argsort(mask_count)
  for k in indices[::-1]:
    # print("{:<8} {:<10} {:<10}".format(k, mask_count[k], mask_count[k] / pos_mask.shape[0] * 100))
    feature_import[k] = mask_count[k] / pos_mask.shape[0] * 100

  return feature_import


def relevant_features_list(feature_import, rel_thr):
  # Feature list
  feature_list = []

  # Iterate over feature_import
  for k,v in feature_import.items():
    if v > rel_thr:
      feature_list.append(k)

  print(len(feature_list), feature_list)
  return feature_list


def del_relevant_features(train_set, test_set, feature_list):
  # Reduced train and test sets
  red_train_set = np.delete(train_set, np.array(feature_list), 1)
  red_test_set = np.delete(test_set, np.array(feature_list), 1)

  # Print sizes of reduced sets
  print(red_train_set.shape)
  print(red_test_set.shape)

  return red_train_set, red_test_set


def train_rrr_model(train_set, train_labels, test_set, test_labels, mask=None):
  tf.compat.v1.random.set_random_seed(123)

  model = TensorflowPerceptron()
  model.fit(train_set, train_labels, hidden_layers=[300,100,50], num_epochs=50, 
            A=mask)
  print(model.score(test_set, test_labels))

  return model


def construct_A_matrix(train_set, feature_list, A_mask=None):
  if A_mask is None:
    # Define A matrix
    A_mask = np.zeros(train_set.shape, dtype=np.uint8)
  
  # Iterate over feature list
  for f in feature_list:
    # Set the coloumn corresponding to f to 1s
    A_mask[:,f] = 1
  
  return A_mask


def flip_bin_mask(bin_mask):
  # Make a copy of bin_mask
  flipped_bin_mask = np.copy(bin_mask)

  # Get boolean arrays for 0s and 1s positions
  zero_pos = (bin_mask == 0)
  one_pos = (bin_mask == 1)

  # Flip 0s and 1s
  flipped_bin_mask[zero_pos] = 1
  flipped_bin_mask[one_pos] = 0

  return flipped_bin_mask


def apply_bin_mask(data, bin_mask):
  return data * bin_mask[:data.shape[0],]


# *************************** Helpers *******************************#
def idx_to_str(idx):
    # Make it 01, 02, etc.
    if idx < 10:
        return '0' + str(idx)

    return str(idx)

def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()

Find important features with the RRR method.

**Prerequisite:** requires execution of the following cells in the following order:


1.   'Normal training' --> Load CSI data of the desired experiment. 
2.   'Train a neural network model with Keras' --> Keras functionality. 
3.   'Right for the Right Reasons (RRR) code taken from the original paper'.
4.   'Logic for applying the RRR method'.







In [ ]:
from json import dumps
import pathlib

# Initilize vars
A_mask=None
Af_mask=None
tf.compat.v1.random.set_random_seed(123)
kr_epochs = 35
tf_epochs = 50
idx = 0
auc_thr = 0.85
res = {}
feature_list = []
log_path = '/content/drive/My Drive/csi-zia/results/rrr-logs'
model_path = '/content/drive/My Drive/csi-zia/results/rrr-models'

# Iterate until convergence
while(1):

  # Train Keras model
  if A_mask is None:
    _, _, _, test_pred, kr_model = nn_model_train_predict(train_data, train_labels, 
                                                          test_data, test_labels, 
                                                          kr_epochs)
  else:
    _, _, _, test_pred, kr_model = nn_model_train_predict(apply_bin_mask(train_data, Af_mask), train_labels, 
                                      apply_bin_mask(test_data, Af_mask), test_labels, 
                                      kr_epochs)
  
  # Get Keras AUC
  kr_auc = roc_auc_score(test_labels, test_pred)

  # Train TensorFlow RRR model
  model = TensorflowPerceptron()
  model.fit(train_data, train_labels, A=A_mask, hidden_layers=[300,100,50], 
            num_epochs=tf_epochs)

  # Get TensorFlow AUC
  tf_auc = roc_auc_score(test_labels, model.predict(test_data))

  # Store results
  res[idx_to_str(idx)] = {'Keras': float(kr_auc), 'TF': float(tf_auc), 
                          'Feat': feature_list, 'Feat_n': len(feature_list)}
  
  # Construct model filename
  model_filename = experiments[0][0].split('/')
  model_filename = model_filename[0].lower() + '-' + \
  model_filename[1].split(' ')[0] + '_' + idx_to_str(idx)

  # Workaround for the office filepath
  if 'office' in model_filename:
    folder = experiments[0][0].split('/')
    folder = folder[0] + '/' + folder[1]
  else:
    folder = experiments[0][0]
  
  # Save Keras model
  kr_model.save(model_path + '/' + folder + '/' + model_filename + '.h5')

  # Get feature importance
  feature_import = get_relevant_features(model, train_data, test_data)

  # Get features that have realtive improtance over 10%
  feature_list = relevant_features_list(feature_import, 10)

  # Construct A_mask for Tensorflow's RRR
  if A_mask is None:
    A_mask = construct_A_matrix(train_data, feature_list)
  else:
    A_mask = construct_A_matrix(train_data, feature_list, A_mask)

  # Construct Af_mask for Keras
  Af_mask = flip_bin_mask(A_mask)

  # Leave while loop
  if kr_auc < auc_thr:
    break
  
  # Increment idx
  idx += 1

# Construct log filename
log_filename = experiments[0][0].split('/')
log_filename = log_filename[0].lower() + '-' + log_filename[1].split(' ')[0]

# Workaround for the office filepath
if 'office' in log_filename:
  folder = experiments[0][0].split('/')
  folder = folder[0] + '/' + folder[1]
else:
  folder = experiments[0][0]

# Create full path to save log file if it does not exist
pathlib.Path(log_path + '/' + folder + '/').mkdir(parents=True, exist_ok=True)

# Save results to a JSON file on Google drive
with open(log_path + '/' + folder + '/' + log_filename + '.json', 'w') as f:
    f.write(dumps(res, default=myconverter, indent=4, sort_keys=True)) 

## Generalizability (cf. Section 6.3 in the paper)

###Train or load the full office model. 

To train the full office model, execute the following cells in the following order before running the next cell:

1.  'Choose the CSI data' --> uncomment 'experiments' for the office. 
2.  'Normal training'.
3.  'Train a neural network model with Keras'.

To load the prevously trained full office model (only an example), execute the cell after the next one.

In [ ]:
# Train office model
train_acc, test_acc, train_pred, test_pred, model = nn_model_train_predict(train_data, train_labels, test_data, test_labels)

In [ ]:
# Check the office's model accuracy and AUC
print('Accuracy: %.4f' % test_acc)
print('AUC: %.4f' % roc_auc_score(test_labels, test_pred))

In [ ]:
from keras.models import load_model
from glob import glob

# Load office model

# Model path (just an example, there is no model)
model_path = '/content/drive/My Drive/csi-zia/full-models/Office/2400 MHz'

# Read model file
model_file = glob(model_path  + '/*.h5', recursive=True)

# Load model
model = tf.keras.models.load_model(model_file[0], 
                                   custom_objects={'leaky_relu': tf.nn.leaky_relu})

### Now load the data of another experiment (other than the office) to evaluate the generalizability, i.e., execute the following cells before running the next one: 

1.  'Choose the CSI data' --> uncomment 'experiments' for the desired experiment. 
2.  'Normal training'.

Set the first two layers in the trained office model to be nontrainable and recompile the model. 

In [ ]:
# Set initial layers to trainable=False
for l in model.layers:
  if l.output_shape[1] > 100 and isinstance(l, keras.layers.Dense):
    l.trainable = False
  print(l.name, l.output_shape, l.trainable)

# Compile the updated model
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

Retrain the last layers of our neural network model on the loaded data from another (i.e., other than the office) experiment. 

In [ ]:
# Perform training
model.fit(train_data, train_labels, verbose=2, epochs=10, batch_size=32)

# Get training and test accuracies
train_loss, train_acc = model.evaluate(train_data, train_labels)
test_loss, test_acc = model.evaluate(test_data, test_labels)

# Make training and test soft predictions
train_pred_soft = model.predict(train_data)
test_pred_soft = model.predict(test_data)

# Convert soft predictions to binary predictions
train_pred = make_bin_predictions(train_pred_soft)
test_pred = make_bin_predictions(test_pred_soft)

Display the results

In [ ]:
print('%s: AUC = %.4f' % (experiments[0][0].split('/')[0], 
                             roc_auc_score(test_labels, test_pred)))

# Advanced attack scenarios (cf. Section 6.5 in the paper).

### Data precollection attack. 

**Description:** use the moving cars model to classify the copresence on the parked car data. To defend against this attack, we employ a number of Right for the Right Reasons (RRR) neural network models. 

**Prerequisite:** Before running the below cell, execute the following cells in the following order:

1.  'Choose the CSI data' --> uncomment 'experiments' for the parking cars. 
2.  'Normal training'.
3.  'Train a neural network model with Keras'.


Load RRR models for the moving cars scenario.

In [6]:
# Load necessary modules
from keras.models import load_model
from glob import glob

# Load scenario RRR models, can be found here: https://doi.org/10.5281/zenodo.5592823
model_path = '/content/drive/My Drive/csi-zia/results/rrr-models/Moving Cars/'

# Read model files
model_files = glob(model_path + experiment[0].split('/')[1] + '/*.h5', 
                   recursive=True)

# List to store models
models = []

# Iterate over models
for mf in model_files:
  # Load a model
  m = tf.keras.models.load_model(mf, 
                                 custom_objects={'leaky_relu': tf.nn.leaky_relu})
  
  # Append to list
  models.append(m)

Apply the loaded RRR models on the parked cars data and display the results.

In [ ]:
# Helper function
def idx_to_str(idx):
    # Make it 01, 02, etc.
    if idx < 10:
        return '0' + str(idx)

    return str(idx)

# Idx to track models
idx = 0

# Iterate over models
for m in models:
  test_pred_soft = m.predict(test_data)

  # Convert soft predictions to binary predictions
  test_pred = make_bin_predictions(test_pred_soft)

  print('%s: AUC = %.4f' % (idx_to_str(idx), 
                            roc_auc_score(test_labels, test_pred)))
  idx += 1

### Increased power attack.

**Description:** use increased power CSI samples from the power scenario to evaluate the performance of our neural network on the unseen data. 

Load office and power data --> this is just to speed things up. Alternatively, one can run the combination of 'Choose the CSI data'  and 'Normal training' two times for the office and power experiments, resulting in the same loaded data.

**Prerequisite:** Before running the below cell, execute the 'Train a neural network model with Keras' cell. 

In [26]:
# Load office data, can be found here: https://doi.org/10.5281/zenodo.5592823
data_path = '/content/drive/My Drive/csi-zia/power-attack/Office/2400 MHz'

# Train and test data (office)
o_train_data = np.load(data_path + '/o_train_data.npy')
o_test_data = np.load(data_path + '/o_test_data.npy')

# Train and test data (office)
o_train_labels = np.load(data_path + '/o_train_labels.npy')
o_test_labels = np.load(data_path + '/o_test_labels.npy')

# Load power data, can be found here: https://doi.org/10.5281/zenodo.5592823
data_path = '/content/drive/My Drive/csi-zia/power-attack/Power/2400 MHz'

# Train and test data (power)
p_train_data = np.load(data_path + '/p_train_data.npy')
p_test_data = np.load(data_path + '/p_test_data.npy')

# Train and test data (power)
p_train_labels = np.load(data_path + '/p_train_labels.npy')
p_test_labels = np.load(data_path + '/p_test_labels.npy')

#### Now there are three options (execute one of the below cells at a time; to play safe, re-execute the above loading cell for each iteration):

(1) Combine training data from both office and power scenarios and make predictions on the power data.

In [ ]:
# Combine office and power data
train_data = np.concatenate((o_train_data, p_train_data), axis=0)
train_labels = np.concatenate((o_train_labels, p_train_labels), axis=0)

# Do the training / make predictions
_, _, _, test_pred, _ = nn_model_train_predict(train_data, train_labels, 
                                               p_test_data, p_test_labels)

Display the results for (1).

In [ ]:
# Display the resulting AUC
print('AUC: %.4f' % roc_auc_score(p_test_labels, test_pred))

(2) Combine training data from both office and power scenarios, **excluding** the high-power samples from the power data (i.e., CSI data from non-colocated devices), and make predictions on the power data.

In [ ]:
# Function to exclude high power CSI data from the power scenario data
def exclude_high_power_data(power_data, power_labels):
  # Indices corresponding to CSI data of colocated devices, i.e., label == 1
  col_csi_idx = []

  # Find indices of CSI measurements corresponding to label 1
  for i in range(len(power_labels)):
    if int(power_labels[i]) == 1:
      col_csi_idx.append(i)
  
  # Get only the data corresponding to colocated devices, i.e., 
  # exclude high-power CSI measurements of non-colocated devices 
  col_csi_data = power_data[col_csi_idx, :]

  return col_csi_data


# Exclude high-power data and prepare the combined training data
exl_p_data = exclude_high_power_data(p_train_data, p_train_labels)

train_data = np.concatenate((o_train_data, exl_p_data), axis=0)
train_labels = np.concatenate((o_train_labels, np.ones(len(exl_p_data))), axis=0)

# Do the training / make predictions
_, _, _, test_pred, _ = nn_model_train_predict(train_data, train_labels, 
                                               p_test_data, p_test_labels)

Display the results for (2).

In [ ]:
# Display the resulting AUC
print('AUC: %.4f' % roc_auc_score(p_test_labels, test_pred))

(3) Combine training data from both office and power scenarios, **using only 10%** of high-power samples from the power data (i.e., CSI data from non-colocated devices), and make predictions on the power data.

In [ ]:
import random

# Function to sample high power CSI data from the power scenario data
def sample_high_power_data(power_data, power_labels, sample_size=0.1):
  # Indices corresponding to CSI data of colocated devices, i.e., label == 1
  col_csi_idx = []
  
  # Find indices of CSI measurements corresponding to label 1
  for i in range(len(power_labels)):
    if int(power_labels[i]) == 1:
      col_csi_idx.append(i)
  
  # Get only the data corresponding to colocated devices, i.e., 
  # exclude high-power CSI measurements of non-colocated devices 
  col_csi_data = power_data[col_csi_idx, :]

  # Get indicies of CSI measurements corresponding to label 0, i.e., non-colocated devices,
  # transmitting with high power
  ncol_csi_idx = list(set([x for x in range(len(power_labels))]) - set(col_csi_idx))

  # Get a random sample size (default 10%) of high-power CSI data, i.e., indices
  ncol_csi_idx_sample = random.sample(ncol_csi_idx, int(len(ncol_csi_idx) * sample_size))

  # Get a sample of high-power CSI data
  ncol_csi_data_sample = power_data[ncol_csi_idx_sample, :]

  return col_csi_data, ncol_csi_data_sample


# Use only a sample of high-power data and prepare the combined training data
exl_p_data, samp_p_data = sample_high_power_data(p_train_data, p_train_labels)

train_data = np.concatenate((o_train_data, exl_p_data, samp_p_data), axis=0)
train_labels = np.concatenate((o_train_labels, np.ones(len(exl_p_data)), np.zeros(len(samp_p_data))), axis=0)

# Do the training / make predictions
_, _, _, test_pred, _ = nn_model_train_predict(train_data, train_labels, 
                                               p_test_data, p_test_labels)

Display the results for (3).

In [ ]:
# Display the resulting AUC
print('AUC: %.4f' % roc_auc_score(p_test_labels, test_pred))